<a href="https://colab.research.google.com/github/tzoung-yu/TrueVolume/blob/main/TrueVolume%E4%B8%8A%E5%B8%82%E4%B8%8A%E6%AB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install twstock
#!pip install mplfinance
#!twstock -U
import twstock
from twstock import Stock
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
#import mplfinance as mpf
import matplotlib as mpl
import yfinance as yf
yfonly=0

#股票代號
stoid="台星科"

def TrueVol(pd):
  df=pd
  opp=df['Open']
  clo=df['Close']
  voo=df['Volume']
  hig=df['High']
  llo=df['Low']
  upl=[]
  dol=[]
  bll=[]
  upc=0.0
  dlc=0.0
  blc=0.0
  lp=0.0
  lgv=1.0
  lgc=0.0
  upordown=0.0
  xvo=[]
  for i in range(len(opp)):
    if clo[i]>lgc:
      lgc=clo[i]
    if(lp==0.0):
      lp=opp[i]
    upordown=0
    if(clo[i]>=opp[i]):
      if(clo[i] >= opp[i] + ((hig[i] - opp[i])/3.8) and clo[i]>=lp):
        upc+=(voo[i]/1000)
        upordown=1
      else:
        if clo[i]<lp or (clo[i] < opp[i] + ((hig[i] - opp[i])/5)):
          dlc+=(voo[i]/1000)
          upordown=-1
    else:
      if(clo[i] <= opp[i]-((opp[i]-llo[i])/3.8) and clo[i]<=lp):
        dlc+=(voo[i]/1000)
        upordown=-1
      else:
        if(clo[i] >= opp[i]-((opp[i]-llo[i])/2) and clo[i]>lp):
          upc+=(voo[i]/1000)
          upordown=1
    upl.append(upc)
    dol.append(dlc)
    blc+=voo[i]/1000
    bll.append(blc/2)
    xvo.append(upordown)
    lp=clo[i]
    if(voo[i]/1000 > lgv):
      lgv=voo[i]/1000

  ema10=clo.ewm(span=9).mean()
  ema30=clo.ewm(span=26).mean()

  exp12     = clo.ewm(span=12, adjust=False).mean()
  exp26     = clo.ewm(span=26, adjust=False).mean()
  macd      = exp12 - exp26
  signal    = macd.ewm(span=9, adjust=False).mean()
  histogram = macd - signal

  gg1=max(upl[-1], dol[-1])
  bbb=0.0
  sco=[]
  x=[]
  vlv=9999/lgv
  vlc=999/lgc
  vlm=2/max(abs(histogram))
  for i in range(len(upl)):
    bbb=((ema10.iloc[i]-ema30.iloc[i])*vlc*666)+((upl[i]-dol[i])*vlv)
    histo=abs(bbb)*vlm*histogram[i]
    if (bbb>0 and histo>0) or (bbb<0 and histo<0):
      bbb = bbb+histo
    else:
      bbb = bbb+(histo*0.7)
    sco.append(bbb)
    x.append(str(i))
  sco=np.array(sco)
  gg2=max(abs(sco))
  gg3=gg1/gg2
  sco=sco*gg3/3.8

  gg3=gg1/max(abs(histogram))
  histogram2 = histogram*gg3/8

  mpl.rcParams["figure.figsize"] = (9.4, 8)

  plt.subplot(311)
  plt.title('Price, EMA')
  plt.plot(ema10)
  plt.plot(ema30)
  plt.plot(clo)
  #plt.plot(histogram2, color='purple')
  #plt.gca().xaxis.set_ticks([])
  plt.grid(color='black')
  #plt.bar(x, height=histogram2, linewidth=0, color='purple')
  #plt.grid(False)

  plt.subplot(312)
  plt.plot(upl,label='Buy')
  plt.plot(dol,label='Sell')
  plt.fill_between(x, histogram2, where=(histogram2<0), color='green', alpha=0.6)
  plt.fill_between(x, histogram2, where=(histogram2>0), color='red', alpha=0.6)
  plt.fill_between(x, sco, where=(sco<0), color='orange', alpha=0.6)
  plt.fill_between(x, sco, where=(sco>0), color='blue', alpha=0.6)
  #plt.plot(sco, label='Score', color='purple')
  #plt.bar(x, height=sco, color='red','linewidth=0)
  plt.gca().xaxis.set_ticks([])
  plt.grid(color='black')
  plt.legend()
  plt.title('TrueVolume')

  xvo = np.array(xvo)
  plt.subplot(313)
  plt.title('Volume')
  #plt.plot(voo)
  plt.fill_between(x, voo, where=(voo>0), color='blue', alpha=0.6)
  plt.fill_between(x, voo, where=(xvo<0), color='green', alpha=0.8)
  plt.fill_between(x, voo, where=(xvo>0), color='red', alpha=0.8)
  plt.gca().xaxis.set_ticks([])
  plt.grid(color='black')

  plt.show()

  '''df = df.set_index("Date")
  apds = [
          mpf.make_addplot(ema10,secondary_y=False,color='b'),
          mpf.make_addplot(ema30,secondary_y=False,color='orange'),
          mpf.make_addplot(histogram,type='bar',width=0.7,panel=1,
                         color='c',alpha=1,secondary_y=False),
         ]

  color=mpf.make_marketcolors(up='red', down='green', inherit=True)
  style1=mpf.make_mpf_style(base_mpf_style='default', gridcolor='black', marketcolors=color)

  mpf.plot(df,type='candle',addplot=apds, figscale=1,figratio=(7,4),title='\n'+stoid+'--EMA & MACD',
                     style=style1,volume=False, panel_ratios=(6,3),returnfig=True)

  plt.show()'''

def keep_first_line(text):
    lines = text.splitlines()
    if lines:
        return lines[0]
    else:
        return ""

thisday = datetime.today()
day_start = thisday - timedelta(days=180)
#day_start = thisday.replace(month=1, day=1)

if yfonly==0:
  codes = twstock.codes
  df = pd.DataFrame.from_dict(codes, orient='index')

  stoname=str(stoid)
  if not stoname.isdigit():
    nn = df[df["name"] == stoname]
    stoid=str(int(nn.code.iloc[0]))

  nn=df[df["code"]==stoid]
  print(stoid, nn.name[0], nn.market[0],nn.group[0])

# 讀取股價資料
if(yfonly==0 and nn.market[0]=='上櫃'):
  print("統計起始日:"+day_start.strftime("%Y/%m"))
  stock = Stock(stoid)
  sto_price = stock.fetch_from(day_start.year, day_start.month)
  sto_attr = ['Date', 'Volume', 'Turnover', 'Open', 'High', 'Low', 'Close', 'Change', 'Transaction']
  data1 = pd.DataFrame(columns = sto_attr, data = sto_price)
  data1 = data1.drop(['Turnover', 'Change', 'Transaction'], axis=1)

if(yfonly==0 and nn.market[0]=='上市'):
  print("統計起始日:"+day_start.strftime("%Y/%m/%d"))
  data1 = yf.download(stoid+'.TW',start=day_start, end=thisday)
  data1 = data1.reset_index(drop=False)

if yfonly==1:
  print(stoid)
  data1 = yf.download(stoid+'.TW',start=day_start, end=thisday)
  data1 = data1.reset_index(drop=False)

df=data1.tail(1)
tt1=str(df["Date"])
tt1= keep_first_line(tt1)
print(tt1[-10:])
tt1=str(round(df["Close"],2))
tt1= keep_first_line(tt1)
print(tt1[6:])

TrueVol(data1)

3265 台星科 上櫃 半導體業
統計起始日:2023/04
